In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time 
import gtts 
import playsound
import mediapipe as mp
from keras.models import load_model


In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils


In [3]:
def mediapipe_detect(img , model):
    img = cv2.cvtColor(img , cv2.COLOR_BGR2RGB)
    img.flags.writeable = False
    results = model.process(img)
    img.flags.writeable = True
    img = cv2.cvtColor(img , cv2.COLOR_RGB2BGR)
    return img , results

In [4]:
def draw_new_landmarks(img , model):
    mp_drawing.draw_landmarks(img , results.pose_landmarks ,mp_holistic.POSE_CONNECTIONS  , mp_drawing.DrawingSpec(color=(80,22,10) , thickness=2 , circle_radius=4) ,mp_drawing.DrawingSpec(color=(80,44,121) , thickness=2 , circle_radius=2))
    mp_drawing.draw_landmarks(img , results.left_hand_landmarks ,mp_holistic.HAND_CONNECTIONS  , mp_drawing.DrawingSpec(color=(121,22,76) , thickness=2 , circle_radius=4) ,mp_drawing.DrawingSpec(color=(121,44,250) , thickness=2 , circle_radius=2))
    mp_drawing.draw_landmarks(img , results.right_hand_landmarks ,mp_holistic.HAND_CONNECTIONS  , mp_drawing.DrawingSpec(color=(245,117,66) , thickness=2 , circle_radius=4) ,mp_drawing.DrawingSpec(color=(245,66,230) , thickness=2 , circle_radius=2))
    

In [5]:
model = load_model('actionss.h5')

In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x , res.y , res.z , res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    lh = np.array([[res.x , res.y , res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x , res.y , res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose,lh,rh])

In [7]:
def make_1080p():
    cap.set(3, 1920)
    cap.set(4, 1080)

def make_720p():
    cap.set(3, 1280)
    cap.set(4, 720)

def make_480p():
    cap.set(3, 640)
    cap.set(4, 480)

def change_res(width, height):
    cap.set(3, width)
    cap.set(4, height)

In [8]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('Data') 

# Actions that we try to detect
actions = np.array(['i love you' , 'i dont understand' , 'thankyou' , 'hello'])

# Thirty videos worth of data
no_sequences = 60

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 30

In [9]:
from scipy import stats

In [10]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [11]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.95

cap = cv2.VideoCapture(0)

make_720p()
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detect(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_new_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                            sound = gtts.gTTS(actions[np.argmax(res)], lang="en")
                            sound.save('hello.mp3')
                            playsound.playsound("hello.mp3")
                            os.remove('hello.mp3')
                            
                    else:
                        sentence.append(actions[np.argmax(res)])
                        sound = gtts.gTTS(actions[np.argmax(res)], lang="en")
                        sound.save('hello.mp3')
                        playsound.playsound("hello.mp3")
                        os.remove('hello.mp3')
                        

            if len(sentence) > 5: 
                sentence = sentence[-5:]

        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (245, 117, 16), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti